# Analysis of the DSAIR model

> A number of replications and extensions of The Anh et al.'s analysis of their DSAIR model.

In [ ]:
#| default_exp analysis_dsair

In [ ]:
#| hide
#| export
from gh_pages_example.conditions import *
from gh_pages_example.methods import *
from gh_pages_example.payoffs import *
from gh_pages_example.types import *
from gh_pages_example.utils import *

import typing

import fastcore.test
from nbdev.showdoc import *
import nptyping
import numpy as np
import pandas
import plotly.express as px
import plotly.graph_objects as go

This notebook contains a number of analyses of different versions of the DSAIR model.

Each analysis involves a number of steps:

1. Create parameter space
2. Run the model
3. Process the results
4. Visualise the results
5. Explain the patterns observed

## Analysis of the baseline DSAIR model

### Create parameter space

In [ ]:
#| export
def fig1_data(b=4, c=1, B=10**4, W=100, β=0.01, Z=100, S=['A','B'], gamma=1):
    """Create parameter grid for the baseline DSAIR model, Figure 1,
    The Anh et al. 2020."""
    namesofvalues=['s','b','c','p','B','W','β','gamma']
    matchingvalues = np.array([[s,b,c,p,B,W,β,gamma]
                               for s in np.arange(1,5.1,0.1)
                               for p in np.arange(0,1.02,0.02)])
    models = {k:v for k, v in zip(namesofvalues, matchingvalues.T)}
    models = {**models,
              'Z':Z, # Z should be a scalar
              'strategy_set':S # S should be a list of strings
             }
    return models

In [ ]:
b=4; c=1; B=10**4; W=100; β=0.1; Z=100; S=["AS", "AU"]; gamma=1
models = fig1_data(b=b, c=c, B=B, W=W, β=β, Z=Z, S=S, gamma=1)

### Run the model

In [ ]:
#| export
results = thread_macro(models,
                       payoffs_sr,
                       payoffs_lr,
                       threshold_society_prefers_safety_dsair,
                       threshold_risk_dominant_safety_dsair,
                       markov_chain,
                      )

In [ ]:
#| hide
{k:v.shape for k,v in results.items() if isinstance(v, np.ndarray)}

### Process the results

Now that we have collected some results, we need to process them so that we can display what we want to.

The general approach I follow is to flatten the `results` dictionary and convert it into a [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) dataframe.

Here, we also compute the risk of an AI related disaster, $p_{risk} = 1 - p$

In [ ]:
#| hide
#| export

flat_results = {k:results[k] for k in ['s','b','c','p','B','W','β']}
flat_results['pr'] = np.round(1 - flat_results['p'], 2)
flat_results['s'] = np.round(flat_results['s'], 2)
for i, strategy in enumerate(["AS", "AU"]):
    flat_results[strategy + "_frequency"] = results['ergodic'][:,i]
    
df = pandas.DataFrame(flat_results)

### Visualise the results

I am using the [Plotly](https://plotly.com/python/) library to visualize our data.



In [ ]:
fig1 = go.Figure(data=go.Heatmap(z=df.AU_frequency, x=df.s, y=df.pr,colorscale=px.colors.sequential.Bluered))
fig1.update_layout(font_size=20, legend_title="AU Frequency", xaxis_title="Development speed, s", yaxis_title="Disaster risk, pr")
fig1.update_annotations(font_size=20)

In [ ]:
s = 1.5
dfplot = df.rename(columns={'AU_frequency':'AU', 'AS_frequency':'AS'})
dfplot = pandas.melt(dfplot[['s', 'pr', 'AU', 'AS']], id_vars=['s', 'pr'], value_vars=['AU', 'AS'])
dfplot = dfplot[dfplot.s==s]
fig2 = px.area(dfplot, x="pr", y="value", color="variable")
fig2.update_layout(font_size=20, legend_title="Strategies", xaxis_title="Disaster risk, pr", yaxis_title="Frequencies")
fig2.update_annotations(font_size=20)

### Explain the patterns observed

# References

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()